In [2]:
%pip install pandas matplotlib openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import re



    

Note: you may need to restart the kernel to use updated packages.


In [3]:


#用pd读取input下的几个excel文件
df_cient_summary = pd.read_excel('./input/生意参谋_客户概况.xlsx')
df_follows_views = pd.read_excel('./input/生意参谋-客户关注分析-核心指标.xlsx')
df_follows_contents = pd.read_excel('./input/生意参谋-内容管理-关注.xlsx')
df_follows_contents_new = pd.read_excel('./input/生意参谋-客户关注分析-上新内容.xlsx')
df_follows_contents_list = pd.read_excel('./input/生意参谋-客户关注分析-清单内容.xlsx')

In [4]:
# 删除哪些没有日期信息数据
df_follows_contents = df_follows_contents[df_follows_contents['关注内容列表'].str.contains(r'\d{4}-\d{2}-\d{2}', regex=True, na=False)]

#把 关注内容列表， 按照空行拆分为一个list，记住是空行
df_follows_contents['关注内容列表2'] = df_follows_contents['关注内容列表'].str.split('\n\t\n')

# 将拆分后的列表转换为 DataFrame
split_columns = df_follows_contents['关注内容列表2'].apply(pd.Series)


## 数据清理

In [5]:
#清理内容列表
#定义一个函数来提取数据
# def extract_data(text):
#     # 拆分文本为几个模块
#     sections = [section for section in text.split('\n\n')]
#     # 提取每个模块的标题和内容
#     title = sections[0].split('：')[1]

#     return {
#         "内容标题"：title,
# }


# # 示例数据
# data = [
#     "高级感衬衫！ | 衬衫怎么搭都好看，让你成为穿搭高手！\nID: 24255682137310\n多品上新\n\n2025-04-11\n20:11:12\n\n猜你喜欢已采纳\n\n已发布\n\n查看数据下线评论管理"
#     # 可以继续添加更多的内容
# ]

# # 创建DataFrame
# df = pd.DataFrame(data, columns=["内容"])

# 定义一个函数来提取信息
def extract_info(text):
    # 拆分文本，按空行分割
    sections = [section.strip() for section in text.split('\n\t\n')]
    #print(sections)
    # 提取各个模块内容
    title = sections[0].strip().split('\n')[0]  # 内容标题, 第一行
    content_id = re.search(r'ID:\s*(\d+)', sections[0]).group(1)  # id
    content_type = sections[0].strip().split('\n')[-1]  # 内容类型，最后一行
    publish_date_match = re.search(r'\d{4}-\d{2}-\d{2}', sections[1])  # 发布日期。 不能用位置判断，因为有字符“定时”
    if publish_date_match:
        publish_date = publish_date_match.group()  # 获取匹配的日期字符串
    else:
        publish_date = None  # 如果没有匹配，设置为 None
    
    publish_time = sections[1].strip().split('\n')[1]  # 发布时间
    adopted = sections[2].strip()  # 是否被采纳
    published = sections[3].strip()  # 是否已发布
    comments = sections[4].strip()  # 查看评论

    # 返回结果
    return {
        "内容标题": title,
        "id": content_id,
        "内容类型": content_type,
        "发布日期": publish_date,
        "发布时间": publish_time,
        "是否被采纳": adopted,
        "是否已发布": published,
        "查看评论": comments
    }

# 应用函数，提取所有行的信息
extracted_data = df_follows_contents["关注内容列表"].apply(extract_info)
print(len(extracted_data))

# 将提取到的数据转换为新的DataFrame
extracted_df = pd.DataFrame(extracted_data.tolist(),dtype=str)
df_follows_contents2 = extracted_df

994


In [11]:

# 统计每个日期的行数； 统计每个日期和内容类型的行数

# 统计每个日期的行数
df_counts = df_follows_contents2.groupby('发布日期').size().reset_index(name='行数')
# 统计每个日期和内容类型的行数，日期列保持唯一值，每种数据类型作为1列，值就是行数

df_counts2 = df_follows_contents2.groupby(['发布日期', '内容类型']).size().reset_index(name='行数')
#df_counts2.to_excel('TEST.xlsx')


# 透视表：行是“发布日期”，列是“内容类型”，值是“行数”
pivot_df = df_counts2.pivot_table(index='发布日期', 
                          columns='内容类型', 
                          values='行数', 
                          fill_value=0)
#pivot_df.to_excel('TEST.xlsx')
#print(pivot_df)


# #把客户关注分析表、df_counts，df_counts2合并， 以df_counts的‘发布日期’作为表索引
df_counts.rename(columns={'行数': '总内容数'}, inplace=True)
df_merge = pd.merge(df_counts, pivot_df, left_on='发布日期', right_on='发布日期', how='left')
df_merge = pd.merge(df_merge, df_follows_views, left_on='发布日期', right_on='日期', how='left')
df_merge = pd.merge(df_merge, df_cient_summary, left_on='发布日期', right_on='日期', how='left')
df_merge = df_merge[::-1]
#df_merge = df_merge.drop(columns=["日期"])


df_merge.to_excel('TEST.xlsx')

#df_analysis = df_merge[['发布日期', '总内容数', '曝光人数', '店铺客户数']].copy()

## 分析店铺客户数、关注内容曝光数、关注内容发布数的关系


In [10]:
# 日期、店铺客户数、关注曝光数，关注发布数

# Ensure df_merge is defined by running the necessary cells above
df_analysis = df_merge[['发布日期', '总内容数', '曝光人数', '店铺客户数']].copy()
df_analysis[['总内容数', '曝光人数', '店铺客户数']] = df_analysis[['总内容数', '曝光人数', '店铺客户数']].replace(',', '', regex=True).apply(pd.to_numeric, errors='coerce')
print(df_analysis.head(10))

cols_to_convert = ['总内容数', '曝光人数', '店铺客户数']
for col in cols_to_convert:
    df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce')

df_analysis['曝光比例'] = (df_analysis['曝光人数'] / df_analysis['店铺客户数'] * 100).round(1)
df_analysis.to_excel('TEST.xlsx')
#以店铺

           发布日期  总内容数    曝光人数     店铺客户数
321  2025-04-20    10     NaN       NaN
320  2025-04-19    10     NaN       NaN
319  2025-04-18    22     NaN       NaN
318  2025-04-17    22  4413.0  121879.0
317  2025-04-16    22  4158.0  138786.0
316  2025-04-15    11  3923.0  109763.0
315  2025-04-14    18  5789.0  111130.0
314  2025-04-13    15  5921.0   96972.0
313  2025-04-12    16  7009.0   75970.0
312  2025-04-11     9  5385.0   56711.0
